<table class="ee-notebook-buttons" align="left"><td>
<a target="_blank"  href="https://colab.research.google.com/github/biagiominio/Remote_sensing_2021/blob/main/R_code_land_cover.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a>
</td><td>
<a target="_blank"  href="https://github.com/biagiominio/Remote_sensing_2021/blob/main/R_code_land_cover.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a></td></table>




# Deforestation in Mato Grosso, Brazil
The state of Mato Grosso is home to the Pantanal complex, the largest floodplain in the world, as well as the second largest ecosystem in the world after the nearby Amazon rainforest. 
The great biodiversity of these areas is threatened by the recent conversion of wetlands and forests into agricultural areas.
A 2006 study found that Brazilian mechanized agriculture increased by more than 3.6 million hectares between 2001 and 2004, growing by more than 540.000 hectares in Mato Grosso alone. Logging for grazing was still the main cause of deforestation at the time, but the contribution of large agricultural clearings, such as soybean plantations, was increasing.

<table>
  <tr>
     <td>Mato Grosso on August 6, 1992</td>
     <td>Mato Grosso on July 28, 2006</td>
  </tr>
  <tr>
    <td><img src="https://eoimages.gsfc.nasa.gov/images/imagerecords/35000/35891/matogrosso_l5_1992219_lrg.jpg" width=500</td>
    <td><img src="https://eoimages.gsfc.nasa.gov/images/imagerecords/35000/35891/matogrosso_ast_2006209_lrg.jpg" width=500</td>
  </tr>
</table>

<font size="2"> [*NASA images*](https://earthobservatory.nasa.gov/images/35891/deforestation-in-mato-grosso-brazil) *created by Jesse Allen, using Landsat data provided by the United States Geological Survey and ASTER data provided courtesy of NASA/GSFC/METI/ERSDAC/JAROS, and U.S./Japan ASTER Science Team. Caption by Michon Scott.*</font> 

In both of these false-color images, red indicates vegetation, and the brighter the red, the denser the vegetation. 
The Rio Peixoto de Azevedo appears pale blue, nearly white, in 1992, perhaps a combination of reflective sediment or sunlight glinting off the water.
The most conspicuous difference between the images is the widespread forest clearing visible as rectangles of gray-beige that had occurred by 2006. 
The most intense areas of clearing appear along roadways.

Use the images shown above to quantify the forest area lost over the past 14 years.
The 1992 image was captured using the Thematic Mapper sensor aboard NASA's Landsat 5 satellite. While the 2006 image was detected by NASA's Terra satellite Advanced Spaceborne Thermal Emission and Reflection Radiometer (ASTER) sensor.


In [1]:
mato_grosso_1992_url <- "https://eoimages.gsfc.nasa.gov/images/imagerecords/35000/35891/matogrosso_l5_1992219_lrg.jpg"
mato_grosso_2006_url <- "https://eoimages.gsfc.nasa.gov/images/imagerecords/35000/35891/matogrosso_ast_2006209_lrg.jpg"

The [download.file](https://www.rdocumentation.org/packages/utils/versions/3.6.2/topics/download.file) function can be used to download a file from the Internet. 
```
download.file(url, destfile)
```
In the function the `url` argument sets the resource to be downloaded and the `destfile` a character string with which the downloaded file is named.

In [2]:
download.file(mato_grosso_1992_url, "Mato_Grosso_1992.jpg")
download.file(mato_grosso_2006_url, "Mato_Grosso_2006.jpg")

## Install R packages
The [install.packages](https://www.rdocumentation.org/packages/utils/versions/3.6.2/topics/install.packages) function is used to download and install packages from CRAN-like repositories.
To install multiple packages at the same time, you define a vector that contains the names of the packages to be installed.

The [raster](https://www.rdocumentation.org/packages/raster/versions/3.4-10) package defines classes and methods for spatial raster data access and manipulation. 

[RStoolbox](https://www.rdocumentation.org/packages/RStoolbox/versions/0.2.6) is an package providing a wide range of tools for your every-day remote sensing processing needs. The available tool-set covers many aspects for remote sensing image processing and analysis such as calculating spectral indices, principal component transformation, unsupervised and supervised classification or fractional cover analyses.

The [ggplot2](https://www.rdocumentation.org/packages/ggplot2/versions/3.3.3) package is a system for declaratively creating graphics. 
It provides a more programmatic interface for specifying what variables to plot, how they are displayed, and general visual properties. 

While the [gridExtra](https://www.rdocumentation.org/packages/gridExtra/versions/2.3/topics/gridExtra-package) package provides a number of user-level functions to work with "grid" graphics, notably to arrange multiple grid-based plots on a page, and draw tables.

In [ ]:
packages <- c("raster", "RStoolbox", "ggplot2", "gridExtra")
install.packages(packages)

The [library](https://www.rdocumentation.org/packages/base/versions/3.6.2/topics/library) function is used to load previously installed packages.
Using the [lapply](https://www.rdocumentation.org/packages/base/versions/3.6.2/topics/lapply) function we simultaneously apply the `library()` function to all elements of the vector containing the packages.



In [ ]:
lapply(packages , library, character.only = TRUE)

## Reading spatial data
To read and view a multilevel object it is common to use the [brick](https://www.rdocumentation.org/packages/raster/versions/3.4-5/topics/brick) function (RasterBrick object). 

Explore the properties of the `mato_grosso_1992` and `mato_grosso_2006` object.



In [5]:
mato_grosso_1992 <- brick( "Mato_Grosso_1992.jpg")
mato_grosso_2006 <- brick( "Mato_Grosso_2006.jpg")

The brick object has three bands that correspond to the following portions of the electromagnetic spectrum:

$\mathsf{B1}$ = Near Infrared; 
$\mathsf{B2}$ = Red;
$\mathsf{B3}$ = Green;

Change the names of a raster object using the [names](https://www.rdocumentation.org/packages/base/versions/3.6.2/topics/names) function.


In [ ]:
names(mato_grosso_1992) <- c("nir", "red", "green")
names(mato_grosso_2006) <- c("nir", "red", "green")
mato_grosso_1992

The two images have a different extension and this could be a problem when comparing pixels or intersections. 

Let's see how to make the extension of the two images equal.

In [7]:
mato_grosso_2006@extent <- mato_grosso_1992@extent

Now the `mato_grosso_1992` image has the same extension as the `mato_grosso_2006` object, but has a different resolution and dimension.

To work around this problem, reduce the resolution of the `mato_grosso_2006` image by enlarging the pixels until they are the same size as the `mato_grosso_1992` object.
To create a new RasterBrick object with a lower resolution use the [aggregate](https://www.rdocumentation.org/packages/raster/versions/3.4-10/topics/aggregate) function of the `raster` package.

In [ ]:
mato_grosso_2006.res <- aggregate(mato_grosso_2006, fun=mean, fact=2)
mato_grosso_2006.res

## Visualization of spatial data
Several generic functions have been implemented for "raster" objects to create maps and other plot types. For example, use [plot](https://www.rdocumentation.org/packages/raster/versions/3.4-5/topics/plot) function to create a map of a raster object.
In addition, individual bands can be combined into a single image by assigning individual layers or bands to one of the three color channels (red, green and blue). 
The [plotRGB](https://www.rdocumentation.org/packages/raster/versions/3.4-5/topics/plotRGB) function can be used to make "true (or false) color images" from multi-band satellite images.

Use the [par](https://www.rdocumentation.org/packages/graphics/versions/3.6.2/topics/par) function to compare the images of Mato Grosso over the two different years.
Export the image using the [pdf](https://www.rdocumentation.org/packages/grDevices/versions/3.6.2/topics/pdf) function to enlarge it better and view the results.

N.B. always use the `dev.off()` function to manage the correct closing and loading of the graphics device.


In [ ]:
par(mfrow=c(1,2))
plotRGB(mato_grosso_1992, r=1, g=2, b=3, stretch="lin")
plotRGB(mato_grosso_2006.res, r=1, g=2, b=3, stretch="lin")
dev.off()

Use the [ggRGB](https://www.rdocumentation.org/packages/RStoolbox/versions/0.2.6/topics/ggRGB) function of the `ggplot2` package to improve the aesthetics of RGB plots.
With [grid.arrange](https://www.rdocumentation.org/packages/pdp/versions/0.7.0/topics/grid.arrange) one can reproduce the behaviour of the base functions `par(mfrow=c(r,c))` specifying either the number of rows or columns.

In [10]:
pdf(file = "/content/compare_mato_grosso.pdf")
p1 <- ggRGB(mato_grosso_1992, r=1, g=2, b=3, stretch="lin")
p2 <- ggRGB(mato_grosso_2006.res, r=1, g=2, b=3, stretch="lin")
grid.arrange(p1, p2, nrow=1)

## Image Classification
The classification of remotely sensed images consists in associating each pixel of the image with a specific class or category (e.g. grassland, forest, water, soil, etc.).
Identification occurs through a process in which the degree of spectral similarity or association is high between pixels of the same category and low between pixels of distinct categories.

It uses *Unsupervised Learning* methods to classify the two images of Mato Grosso in Brazil. Define a number of classes k to estimate the forest area lost from 1992 to 2006 or changes in land cover.
[K-means](https://en.wikipedia.org/wiki/K-means_clustering) clustering is one of the simplest and popular unsupervised learning algorithms.
Typically, unsupervised algorithms make inferences from datasets using only input vectors without referring to known, or labeled, outcomes.

The `RStoolbox` package provides the [unsuperClass](https://www.rdocumentation.org/packages/RStoolbox/versions/0.2.6/topics/unsuperClass) function which allows you to apply the k-means algorithm to any Raster Object.

Furthermore, by setting the same seed with the [set.seed](https://www.rdocumentation.org/packages/simEd/versions/2.0.0/topics/set.seed) function (a random number generator) we always select the same initial pixels during the construction phase of the n-clusters in order to always show the same output.


In [11]:
set.seed(1234)
mato_grosso_1992.class <- RStoolbox::unsuperClass(mato_grosso_1992, nClasses=3)

Use function [colorRampPalette](https://www.rdocumentation.org/packages/dichromat/versions/1.1/topics/colorRampPalette) to create a continuous color scale divided into n-levels.


In [ ]:
cl <- colorRampPalette(c('yellow3','royalblue','darkred'))(3)
plot(mato_grosso_1992.class$map, col=cl)

The spatial patterns identified represent the main land cover classes:
* yellow indicates agricultural areas (cluster 1);
* blue indicates waterways and wetlands cultivated with rice (cluster 2);
* and finally red indicates forest (cluster 3).

Typically, unsupervised algorithms make inferences from datasets using only input vectors without referring to known or labeled results. Therefore the content of each class should be validated in the field or compared with other data demonstrating belonging to the chosen land cover type. This step is very important and is useful for defining the accuracy of the classification model used.

Repeat the same procedure for the 2006 image.

In [ ]:
set.seed(23489)
mato_grosso_2006.res.class <- RStoolbox::unsuperClass(mato_grosso_2006.res, nClasses=3)
cl2 <- colorRampPalette(c('yellow3','royalblue','darkred'))(3)
plot(mato_grosso_2006.res.class$map, col=cl2)

Also in this case apply the same number of clusters and type of colors to facilitate the visual comparison.

## How much forest area has been lost?
Raster images are made up of pixels whose number and extent are known to us. Calculate the pixel frequency for each type of land cover or area knowing that each pixel has the same 30 m span.

In statistics the frequency (or absolute frequency) of an event $i$ is the number $n_{i}$ of times the observation occurred/recorded in an experiment or study. Generate a frequency table using the [freq](https://www.rdocumentation.org/packages/questionr/versions/0.7.4/topics/freq) function.
Multiply the obtained matrix by the spatial resolution of each pixel (30 m) to obtain the absolute area.

In [ ]:
pixel_resolution <- 30*30 # square meters
absolute_frequency_1992 <- freq(mato_grosso_1992.class$map) 
area_km2 <- absolute_frequency_1992[,2] * pixel_resolution / 10^6 # from meters to square kilometers
area_km2

# Agricolture = 350.88 km^2
# Wetland = 179.86 km^2
# Forest = 1089.26 km^2

To compare the surface variations between the two images it is necessary to normalize the absolute area, dividing by the total area. This means calculating the relative frequency (or empirical probability) of the pixels within each land cover class. At this point you can simply divide the number of counts or the absolute frequency by the total number of events:

$$f_{i}={\frac{n_{i}{r_s}}{{N}{r_s}}}={\frac{n_{i}}{\sum_{j}n_{j}}}$$

where $r_s$ is the spatial resolution of each pixel. 

Now use the simplified formula to calculate the normalized area or relative frequency.



In [ ]:
total_pixel_1992 <- sum(absolute_frequency_1992[,2]) 
relative_frequency_1992 <- absolute_frequency_1992[,2]/total_pixel_1992

Rename the three clusters by assigning each of them their respective land cover.

* **Agricolture** = cluster **1**
* **Wetlands** = cluster **2**
* **Forest** = cluster **3**


Use the [data.frame](https://www.rdocumentation.org/packages/base/versions/3.6.2/topics/data.frame) function to create a data frame, a matrix-like structure whose columns may be of differing types (numeric, logical, factor and character and so on).


In [ ]:
labels <- c("Agriculture", "Wetlands", "Forest")
data_freq_1992 <- data.frame(cover = labels, count = relative_frequency_1992)
data_freq_1992

In 1992 the Mato Grosso area was mainly covered by woods with a coverage of about 67%. Only 22% was used for agriculture.

Apply the same procedure to the 2006 image.

In [ ]:
pixel_resolution <- 30*30 # square meters
absolute_frequency_2006 <- freq(mato_grosso_2006.res.class$map) 
area_km2 <- absolute_frequency_2006[,2] * pixel_resolution / 10^6 # from meters to square kilometers
area_km2

# Agricolture = 631.47 km^2
# Wetland = 361.03 km^2
# Forest = 627.50 km^2

Normalize the area for later comparison.

In [ ]:
total_pixel_2006 <- sum(absolute_frequency_2006[,2]) 
relative_frequency_2006 <- absolute_frequency_2006[,2]/total_pixel_2006
labels <- c("Agriculture", "Wetlands", "Forest")
data_freq_2006 <- data.frame(cover = labels, count = relative_frequency_2006)
data_freq_2006

In 2006 the frequencies changed in favor of the agricultural class with a coverage of 39%. Consider that the class called `"Wetlands"` includes both areas cultivated with rice and waterways. While the forest area has shrunk by a third.

Use the [cbind](https://www.rdocumentation.org/packages/base/versions/3.6.2/topics/cbind) function to join the two previous data frames and compare the relative frequencies for each land cover type.

NB: `cbind.data.frame` defines the new object as a data frame and not as a matrix, while the [round](https://www.rdocumentation.org/packages/base/versions/3.6.2/topics/Round) function rounds the numeric values and makes the table more readable.



In [ ]:
join_data <- cbind.data.frame(cover=labels, 
                    freq_1992=round(data_freq_1992$count,4), 
                    freq_2006=round(data_freq_2006$count,4))
join_data

Compare land cover changes from 1992 to 2006 in Mato Grosso and quantify the forest loss that occurred. Visualize the frequencies for each land cover type using an appropriate diagram, such as a bar charts (classes are categories and not ranges). 

Create new graphs using the [ggplot](https://www.rdocumentation.org/packages/ggplot2/versions/3.3.3/topics/ggplot) function.

Create a new graph space using the [ggplot](https://www.rdocumentation.org/packages/ggplot2/versions/3.3.3/topics/ggplot) function and the [geom_bar](https://www.rdocumentation.org/packages/ggplot2/versions/3.3.3/topics/geom_bar) function to add bars to the graph.



In [ ]:
# pdf(file = "/content/compare_hist.pdf")
p_1992 <- ggplot(join_data, aes(x=cover, y=freq_1992)) +
          geom_bar(aes(fill=cover), stat="identity") + 
          scale_fill_manual(values=c("yellow3", "darkred", "royalblue")) +
          ylim(0,0.68) +
          theme_classic()

p_2006 <- ggplot(join_data, aes(x=cover, y=freq_2006)) + 
          geom_bar(aes(fill=cover), stat="identity") + 
          scale_fill_manual(values=c("yellow3", "darkred", "royalblue")) +
          ylim(0,0.68) +
          theme_classic()

grid.arrange(p_1992, p_2006, nrow=1)
dev.off()

Take a look at the online version of the book ["ggplot2: Elegant Graphics for Data Analysis"](https://ggplot2-book.org/) published by Springer to learn the grammar of the graphics used by [ggplot2](https://ggplot2.tidyverse.org/).